In [40]:
import pandas as pd
import re as re
import gensim
import nltk
from nltk.tokenize.treebank import TreebankWordTokenizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras import regularizers
from keras.layers import Embedding

In [3]:
interactions  = pd.read_csv('RAW_interactions.csv',index_col=0)# read csv into ratings_df dataframe
interactions.shape

(1132367, 4)

In [10]:
#interactions  = pd.read_csv('RAW_interactions.csv',index_col=0)# read csv into ratings_df dataframe
interactions=interactions.sample(10000)
interactions_df = pd.DataFrame(interactions)
interactions_df.head()

,recipe_id,date,rating,review
user_id,,,,
218535,239512,2014-12-29,1,Really don&#039;t know how to say this but thi...
314785,82778,2007-07-12,3,I liked this soup creamy soup. But I don't car...
2001462233,150384,2017-05-09,5,"I've never had Panera soup, but this recipe ma..."
1803521092,386789,2014-12-18,3,"A fair recipe. However, EPIC FAIL, on the note..."
279989,148010,2008-09-28,5,This was really tasty. I couldn't find the fl...


In [29]:
for i in range(len(interactions_df['review'].values.tolist())):
    if str(interactions_df['review']) == 'nan,':
        print('YES')

In [5]:
def depure_data(data):
    
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    
    # Remove Emails
    data = re.sub('\S*@\S*\s?', '', data)

    # Remove new line characters
    data = re.sub('\s+', ' ', data)

    # Remove distracting single quotes
    data = re.sub("\'", "", data)
        
    return data

In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [7]:
def detokenize(text):
    return nltk.tokenize.treebank.TreebankWordDetokenizer().detokenize(text)

In [43]:
temp = []
data_to_list = interactions_df['review'].values.tolist()
ratings = interactions_df['rating'].values.tolist()
remove_ratings = []
for i in range(len(data_to_list)):
    if str(data_to_list[i]) != 'nan':
        temp.append(depure_data(data_to_list[i]))
    else:
        print('NAN I Is: ', i, "Review is: ", str(data_to_list[i]))
        remove_ratings.append(i)
data_words = list(sent_to_words(temp))
data = []
for i in range(len(data_words)):
    data.append(detokenize(data_words[i]))

NAN I Is:  1984 Review is:  nan
NAN I Is:  9743 Review is:  nan


In [41]:
# Top 5 ratings before preprocessing
print(data_to_list[:5])

['Really don&#039;t know how to say this but this is not good - bland and blah.', "I liked this soup creamy soup. But I don't care for the sour cream in it; I think it takes away from the mushroom flavor.  I even used just 1 c of lite sour cream, but that seemed too much. Next time, I'll omit sour cream entirely.  (My 4 yr old surprisingly likes this soup too.)", "I've never had Panera soup, but this recipe makes a delicious soup. Only changes I made were 2 cloves of garlic sauted with the onion, whole milk instead of half/half, and I only added about five ounces of cheese as opposed to eight. Seemed like 8 ounces was a bit much for such a small volume of soup. The soup thickened up beautifully. Creamy and thick without being overly rich. For those who complain that this has no flavor, I am baffled. My first guess is lack of salt. The soup does require a good bit of salt and pepper, but that's the nature of any recipe. You salt as you go, and add as much as you think it needs to bring 

In [44]:
# Top 5 ratings after preprocessing
print(data[:5])

['really don know how to say this but this is not good bland and blah', 'liked this soup creamy soup but dont care for the sour cream in it think it takes away from the mushroom flavor even used just of lite sour cream but that seemed too much next time ill omit sour cream entirely my yr old surprisingly likes this soup too', 'ive never had panera soup but this recipe makes delicious soup only changes made were cloves of garlic sauted with the onion whole milk instead of half half and only added about five ounces of cheese as opposed to eight seemed like ounces was bit much for such small volume of soup the soup thickened up beautifully creamy and thick without being overly rich for those who complain that this has no flavor am baffled my first guess is lack of salt the soup does require good bit of salt and pepper but thats the nature of any recipe you salt as you go and add as much as you think it needs to bring out the flavor my second guess is using bland supermarket cheese used go

In [16]:
# Remove those datapoints where review was 'nan'
ratings = interactions_df['rating'].values.tolist()
final_ratings = []
for i in range(len(ratings)):
    if i not in remove_ratings:
        final_ratings.append(ratings[i])

In [17]:
len(final_ratings)

9998

In [19]:
len(data)

9998

In [22]:
# Create word embeddings from the preprocessed text data

max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
reviews = pad_sequences(sequences, maxlen=max_len)
print(reviews)

[[   0    0    0 ...  455    2 3247]
 [   0    0    0 ...    4  132   56]
 [   0    0    0 ...   83    4   40]
 ...
 [   0    0    0 ...   20    3   40]
 [   0    0    0 ...    4  101  122]
 [   0    0    0 ...   19    6   81]]


In [38]:
# embedding_layer = Embedding(1000, 64)   ### Sample embedding layer

In [37]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(data,final_ratings, test_size = 0.3)

In [36]:
model = Sequential()
model.add(layers.Embedding(max_words, 40, input_length=max_len))
model.add(layers.Bidirectional(layers.LSTM(20,dropout=0.6)))
model.add(layers.Dense(3,activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=5,validation_data=(X_test, y_test))

#Validating model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('Model accuracy: ',test_acc)

model_path = '{}/{}/001'.format(model_dir, 'tensorflow-sentiment-predictions')
tf.saved_model.save(model, model_path)

Epoch 1/5


ValueError: in user code:

    File "/home/aomkar/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/aomkar/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/aomkar/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/aomkar/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/home/aomkar/anaconda3/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/aomkar/anaconda3/lib/python3.8/site-packages/keras/engine/input_spec.py", line 232, in assert_input_compatibility
   

In [39]:
model0 = Sequential()
model0.add(layers.Embedding(max_words, 15))
model0.add(layers.SimpleRNN(15,return_sequences=True))
model0.add(layers.SimpleRNN(15))
model0.add(layers.Dense(3,activation='softmax'))
model0.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=5,validation_data=(X_test, y_test))

Epoch 1/5


ValueError: in user code:

    File "/home/aomkar/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/aomkar/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/aomkar/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/aomkar/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/home/aomkar/anaconda3/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/aomkar/anaconda3/lib/python3.8/site-packages/keras/engine/input_spec.py", line 232, in assert_input_compatibility
   